In [19]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

path = "C:/Data/chinaUnicom/"
train_data = pd.read_csv(path + "train/train_.csv")
test_data = pd.read_csv(path + "test/test_.csv")

In [21]:
train_data.describe()

,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,...,local_trafffic_month,local_caller_time,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee,current_service
count,612647.000000,612647.000000,612647.000000,612647.000000,612647.000000,612647.000000,612647.000000,612647.000000,612647.000000,612647.000000,...,612647.000000,612647.000000,612647.000000,612647.000000,612647.000000,612647.000000,612647.000000,612647.000000,6.126470e+05,6.126470e+05
mean,2.748850,0.035844,42.831485,97.412344,102.702115,96.149908,102.870922,1159.342848,0.426862,1.491526,...,5828.271412,59.028263,42.500550,84.485437,1.277553,31.727148,0.119199,0.139465,5.000722e+06,9.225263e+07
std,1.438614,0.185902,45.367991,89.426338,97.642109,93.311736,101.235509,2754.769136,0.494622,2.967213,...,11296.540492,115.763748,110.368447,137.038245,0.510085,12.289361,0.380601,0.491295,2.798663e+09,4.218052e+06
min,1.000000,0.000000,1.000000,0.000000,-287.300000,-276.030000,-420.270000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,8.901625e+07
25%,1.000000,0.000000,10.000000,46.200000,47.200000,42.000000,44.900000,0.000000,0.000000,0.000000,...,95.324635,0.000000,0.000000,0.000000,1.000000,24.000000,0.000000,0.000000,0.000000e+00,8.995017e+07
50%,4.000000,0.000000,21.000000,72.630000,76.000000,69.000000,74.100000,139.236942,0.000000,0.000000,...,1262.557972,14.066667,0.000000,29.150000,1.000000,29.000000,0.000000,0.000000,0.000000e+00,9.006334e+07
75%,4.000000,0.000000,64.000000,116.000000,126.000000,119.750000,129.200000,1311.389001,1.000000,1.000000,...,5185.147894,67.516667,34.475000,123.966667,2.000000,39.000000,0.000000,0.000000,0.000000e+00,9.015595e+07
max,4.000000,1.000000,274.000000,5940.830000,5825.570000,6965.500000,5141.270000,159057.397800,1.000000,12.000000,...,484365.746300,6461.050000,5139.483333,16454.383330,2.000000,99.000000,3.000000,37.000000,1.762170e+12,9.999983e+07


In [ ]:
def increaseFeatures(train):
    # 费用
    train['avg_total_fee']=(train['1_total_fee']+train['2_total_fee']+train['3_total_fee']+train['3_total_fee']).astype(float)/4.0
    train['sum_total_fee'] = (train['1_total_fee']+train['2_total_fee']+train['3_total_fee']+train['3_total_fee']).astype(float)
    train['pay_num_per'] = train['pay_num'].astype(float)/train['pay_times']
    train['contract_timex_feex_over_bill'] = (train['avg_total_fee']*train['contract_time']*train['many_over_bill']).astype(float)
    train['contract_online_time_rate'] = (train['contract_time']/train['online_time']).astype(float)
    # 流量
    train['sum_traffic_month'] = train['month_traffic'] + train['local_trafffic_month']+1
    train['out_local_traffic_rate'] = train['month_traffic'].astype(float)/train['sum_traffic_month']
    train['local_sum_traffic_rate'] = train['local_trafffic_month'].astype(float)/train['sum_traffic_month']
    train['last_month_sum_traffic_rate'] = train['last_month_traffic'].astype(float)/train['sum_traffic_month']
    # 通话时间
    train['sum_caller_receive_time'] = train['local_caller_time']+train['service1_caller_time']+train['service2_caller_time']+1
    train['sum_caller_time'] = train['local_caller_time']+train['service1_caller_time']
    train['caller_receive_time_rate'] = train['sum_caller_time'].astype(float)/(train['sum_caller_receive_time'])
    # 流量+通话时间+费用
    train['month_traffic_total_fee_rate'] = train['sum_traffic_month'].astype(float)/train['sum_caller_receive_time']
    train['month_traffic_caller_rate'] = train['sum_traffic_month'].astype(float)/(train['1_total_fee']+1)
    train['caller_time_total_fee_rate'] = train['sum_caller_receive_time'].astype(float)/(train['1_total_fee']+1)
    #省内、本地流量通话
    train['local_trafficx_local_caller'] = train['local_trafffic_month']*train['local_caller_time'].astype(float)
    train['month_trafficx_service1_call'] = train['month_traffic']*train['service1_caller_time'].astype(float)
    # CosLog
    
    return train

train_data = increaseFeatures(train_data)
# test_data = increaseFeatures(test_data)

In [18]:
cosFeatures = ['online_time', '1_total_fee','2_total_fee', '3_total_fee', '4_total_fee', 
       'month_traffic', 'contract_time','pay_times', 'pay_num','month_trafficx_service1_call',
       'last_month_traffic', 'local_trafffic_month','local_caller_time','service1_caller_time',  
       'service2_caller_time','age','avg_total_fee','pay_num_per','caller_time_total_fee_rate', 
       'contract_timex_feex_over_bill','contract_online_time_rate', 'sum_traffic_month',
       'out_local_traffic_rate','local_sum_traffic_rate','last_month_sum_traffic_rate',  
       'sum_caller_receive_time','sum_caller_time', 'caller_receive_time_rate',
       'month_traffic_total_fee_rate', 'month_traffic_caller_rate', 'local_trafficx_local_caller']
# train_data[cosFeatures].mean()
(train_data[cosFeatures]-train_data[cosFeatures].min())/(train_data[cosFeatures].max()-train_data[cosFeatures].min())
# np.log(train_data[cosFeatures].max())

      

,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,contract_time,pay_times,pay_num,month_trafficx_service1_call,...,sum_traffic_month,out_local_traffic_rate,local_sum_traffic_rate,last_month_sum_traffic_rate,sum_caller_receive_time,sum_caller_time,caller_receive_time_rate,month_traffic_total_fee_rate,month_traffic_caller_rate,local_trafficx_local_caller
0,0.307692,0.049818,0.095454,0.078993,0.128934,2.397634e-02,0.698113,0.007937,5.416191e-03,0.000000e+00,...,1.074736e-02,0.732455,0.267358,3.038841e-08,0.040836,0.016363,0.160537,1.596114e-05,1.821385e-04,1.146229e-03
1,0.032967,0.044640,0.089729,0.066910,0.106943,0.000000e+00,0.018868,0.015873,5.415469e-03,0.000000e+00,...,1.297625e-01,0.000000,0.999986,0.000000e+00,0.036148,0.090102,0.998476,2.176278e-04,2.452874e-03,1.149501e-01
2,0.040293,0.007491,0.058483,0.047646,0.086607,1.633622e-02,0.018868,0.023810,9.024277e-04,0.000000e+00,...,9.359951e-03,0.573014,0.426770,0.000000e+00,0.007656,0.004188,0.217738,7.366801e-05,1.035342e-03,4.078370e-04
3,0.487179,0.014804,0.060315,0.048613,0.091444,6.214364e-03,0.018868,0.000000,1.805036e-03,1.289106e-03,...,4.081630e-03,0.499720,0.499779,7.291432e-10,0.017381,0.024882,0.572420,1.421798e-05,2.310067e-04,6.769516e-04
4,0.304029,0.053366,0.098379,0.098258,0.149836,3.700426e-02,0.471698,0.087302,1.805253e-02,3.617360e-03,...,1.215156e-02,0.999836,0.000000,2.169183e-08,0.032296,0.005313,0.065884,2.280923e-05,1.922823e-04,0.000000e+00
5,0.164835,0.006060,0.052888,0.043089,0.082256,1.370596e-03,0.245283,0.000000,5.413844e-04,0.000000e+00,...,9.001605e-04,0.498859,0.498857,4.059843e-08,0.001777,0.002215,0.483554,2.980898e-05,1.226933e-04,2.429971e-05
6,0.040293,0.005656,0.052349,0.043462,0.083784,0.000000e+00,0.018868,0.000000,9.024277e-04,0.000000e+00,...,0.000000e+00,0.000000,0.000000,0.000000e+00,0.008632,0.021516,0.993166,1.426314e-08,2.944514e-07,0.000000e+00
7,0.245421,0.033687,0.057878,0.047301,0.087524,0.000000e+00,0.018868,0.023810,5.415469e-03,0.000000e+00,...,7.060943e-03,0.000000,0.999710,0.000000e+00,0.000054,0.000131,0.460791,3.754691e-03,1.766961e-04,2.257786e-05
8,0.395604,0.019526,0.065976,0.054136,0.096425,3.998219e-03,0.245283,0.007937,3.610252e-03,0.000000e+00,...,2.625959e-03,0.499598,0.499621,3.107449e-08,0.034104,0.037975,0.445991,4.670905e-06,1.130185e-04,1.215370e-03
9,0.102564,0.002693,0.050181,0.040824,0.078768,1.376633e-08,0.018868,0.015873,1.624514e-03,0.000000e+00,...,9.042002e-09,0.002180,0.002180,5.768994e-06,0.004123,0.005139,0.492819,2.993654e-08,6.079718e-07,5.663414e-10


In [10]:
for column in train_data.columns:
    print(train_data[column].describe())

count    612647.000000
mean          2.748850
std           1.438614
min           1.000000
25%           1.000000
50%           4.000000
75%           4.000000
max           4.000000
Name: service_type, dtype: float64
	
count    612647.000000
mean          0.035844
std           0.185902
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: is_mix_service, dtype: float64
	
count    612647.000000
mean         42.831485
std          45.367991
min           1.000000
25%          10.000000
50%          21.000000
75%          64.000000
max         274.000000
Name: online_time, dtype: float64
	
count    612647.000000
mean         97.412344
std          89.426338
min           0.000000
25%          46.200000
50%          72.630000
75%         116.000000
max        5940.830000
Name: 1_total_fee, dtype: float64
	
count    612647.000000
mean        102.702115
std          97.642109
min        -287.300000
25%          47.200000
5

count    6.126470e+05
mean     2.813188e+05
std      1.123952e+06
min      0.000000e+00
25%      0.000000e+00
50%      1.938007e+04
75%      1.643163e+05
max      1.312821e+08
Name: local_trafficx_local_caller, dtype: float64
	
count    6.126470e+05
mean     5.813524e+04
std      4.271768e+05
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      5.711115e+07
Name: month_trafficx_service1_call, dtype: float64
	


In [ ]:
train = train_data.drop(['current_service','user_id','sum_total_fee','net_service','complaint_level','former_complaint_num','former_complaint_fee'],axis=1)
original_feature_new = train.columns
allFold_data = train[original_feature_new]